In [ ]:
"""
Created on Thu Jan 20 15:40 2021

script to plot Figure 8b and 8c
Based on data produced with "apply_param_PIGL_dutrieux_BedMachine.ipynb"
author: @claraburgard
"""

In [ ]:
import xarray as xr
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import assess_param_funcs.useful_functions as uf
import cmocean
import seaborn as sns

import distributed

import matplotlib as mpl
import glob

In [ ]:
sns.set_context('paper')

In [ ]:
client = distributed.Client(n_workers=8, dashboard_address=':8795', local_directory='/tmp', memory_limit='6GB')

In [ ]:
BedMachine_folder = ''#'_reduced5'

In [ ]:
home_path='/bettik/burgardc/'

In [ ]:
inputpath_data='/bettik/burgardc/DATA/BASAL_MELT_PARAM/raw/BEDMACHINE_RTOPO/'
inputpath_mask = home_path+'/DATA/BASAL_MELT_PARAM/interim/ANTARCTICA_IS_MASKS/BedMachine_for_comparison'+BedMachine_folder+'/'
outputpath_melt = home_path+'DATA/BASAL_MELT_PARAM/processed/MELT_RATE/BedMachine_for_comparison'+BedMachine_folder+'/'
inputpath_plumes = '/bettik/burgardc/DATA/BASAL_MELT_PARAM/interim/PLUMES/BedMachine_for_comparison'+BedMachine_folder+'/'
inputpath_boxes = '/bettik/burgardc/DATA/BASAL_MELT_PARAM/interim/BOXES/BedMachine_for_comparison'+BedMachine_folder+'/'
plot_path = '/bettik/burgardc/PLOTS/BedMachine_stuff/'

MASK FILES

In [ ]:
# make the domain a little smaller to make the computation even more efficient - file isf has already been made smaller at its creation
map_lim = [-3000000,3000000]

In [ ]:
file_isf_orig = xr.open_dataset(inputpath_mask+'BedMachine'+BedMachine_folder+'_isf_masks_and_info_and_distance_new.nc')
file_isf = file_isf_orig.sel(Nisf=[66])
file_isf['isf_name'] = file_isf['isf_name'].astype(str)

In [ ]:
plume_charac = xr.open_dataset(inputpath_plumes+'BedMachine'+BedMachine_folder+'_plume_characteristics.nc')

MELT FILES

In [ ]:
melt_param_files = list(sorted(glob.glob(outputpath_melt+'melt_rates_2D*.nc')))

In [ ]:
param_list = []
for mfilename in melt_param_files:
    if BedMachine_folder == '_reduced5':
        paramname = mfilename[108:157].split('.')[0].split('_')[:-2]
    else:
        paramname = mfilename[99:157].split('.')[0].split('_')[:-2]
    mparam = '_'.join(paramname)
    param_list.append(mparam)

In [ ]:
param_list

In [ ]:
ds_melt_param = xr.open_mfdataset(melt_param_files, concat_dim='new_param', combine='nested', coords='minimal',compat='override', chunks={'x': 2000, 'y': 2000})


In [ ]:
melt_m_ice_per_y = ds_melt_param['melt_m_ice_per_y'].drop('param').drop('option').drop('depth').drop('profile_domain')
melt_m_ice_per_y  = melt_m_ice_per_y.rename({'new_param': 'param'})
melt_m_ice_per_y = melt_m_ice_per_y.assign_coords(param=param_list)

MAKE TIME MEAN

In [ ]:
time_mean_melt = melt_m_ice_per_y.mean('time')

In [ ]:
melt_m_ice_per_y_PIGL_pattern = time_mean_melt.where(np.isfinite(time_mean_melt), drop=True)

In [ ]:
melt_m_ice_per_y_PIGL_pattern.to_netcdf(outputpath_melt+'melt_rates_PIGL_dutrieux_time_mean_pattern'+BedMachine_folder+'.nc')

PREPARE PLOT STUFF

In [ ]:
grounded_msk03 = file_isf['ground_mask'].where(file_isf['ground_mask']==0,3)
grounded_msk = (grounded_msk03.where(grounded_msk03!=3,1)-1)*-1

In [ ]:
file_isf_mask = file_isf['ISF_mask'].where(file_isf['ISF_mask']==file_isf.Nisf).sum('Nisf')
icesheet_msk_0inf = file_isf_mask.where(file_isf_mask!=1,0)
icesheet_msk = icesheet_msk_0inf.where(icesheet_msk_0inf < 1, 1)

In [ ]:
box_charac_all_2D = xr.open_dataset(inputpath_boxes + 'BedMachine'+BedMachine_folder+'_boxes_2D.nc')
box_charac_all_1D = xr.open_dataset(inputpath_boxes + 'BedMachine'+BedMachine_folder+'_boxes_1D.nc')
box_loc_config2 = box_charac_all_2D['box_location'].sel(box_nb_tot=box_charac_all_1D['nD_config'].sel(config=2))
box1 = box_loc_config2.where(box_loc_config2==1).isel(Nisf=0).drop('Nisf')
box1_msk = box1.where(box1==1,0)

PLOT FIGURE 8B

In [ ]:
kisf = 66

ref_melt_isf = melt_m_ice_per_y_PIGL_pattern.isel(param=0)
#cmaph = cmocean.cm.thermal
cmaph = mpl.cm.Reds

levs = np.arange(0,71,10)#[245, 250, 255, 260, 265, 270, 275, 280, 285, 290, 310, 340]

if kisf == 11:
    ymax = ref_melt_isf.y.min() - 80000
    xmin = ref_melt_isf.x.min() + 10000
elif kisf == 10:
    ymax = ref_melt_isf.y.min() - 80000
    xmin = ref_melt_isf.x.min() + 5000
elif kisf == 31:
    ymax = ref_melt_isf.y.min() - 20000
    xmin = ref_melt_isf.x.min() + 3000 
elif kisf == 44:
    ymax = ref_melt_isf.y.min() - 20000
    xmin = ref_melt_isf.x.min() + 3000 
elif kisf == 48:
    ymax = ref_melt_isf.y.min() - 15000
    xmin = ref_melt_isf.x.min() + 3000   
elif kisf == 66:
    ymax = ref_melt_isf.y.min() - 15000
    xmin = ref_melt_isf.x.min() + 1000   
    
#min_maxval = (abs(ref_melt_isf)).max().values
min_maxval=50

max_list = []

fig, ax = plt.subplots(1,7,sharex=True,sharey=True)
fig.set_size_inches(8.25*1.7, 8.25/4.5)
for i in range(len(ax)):
    if i==0:
        melt_m_ice_per_y_PIGL_pattern.sel(param='linear_local').plot.contourf(ax=ax[i],cmap=cmaph,vmin=0,vmax=min_maxval,levels=levs,add_colorbar=False)#,vmin=minval, vmax=maxval,zorder=0)  
        max_lin_Ant = melt_m_ice_per_y_PIGL_pattern.sel(param='linear_local').quantile(0.9).values
        max_list.append(max_lin_Ant)
        ax[i].text(xmin,ymax,'Melt 90pctl = '+str(np.round(max_lin_Ant,1)))
    elif i==1:
        melt_m_ice_per_y_PIGL_pattern.sel(param='quadratic_local').plot.contourf(ax=ax[i],cmap=cmaph,vmin=0,vmax=min_maxval,levels=levs,add_colorbar=False)#,vmin=minval, vmax=maxval,zorder=0)  
        max_quad_Ant = melt_m_ice_per_y_PIGL_pattern.sel(param='quadratic_local').quantile(0.9).values
        max_list.append(max_quad_Ant)
        ax[i].text(xmin,ymax,'Melt 90pctl = '+str(np.round(max_quad_Ant,1)))
    elif i==2:
        melt_m_ice_per_y_PIGL_pattern.sel(param='quadratic_mixed_locslope').plot.contourf(ax=ax[i],cmap=cmaph,vmin=0,vmax=min_maxval,levels=levs,add_colorbar=False)#,vmin=minval, vmax=maxval,zorder=0)  
        max_quad_semi_loc = melt_m_ice_per_y_PIGL_pattern.sel(param='quadratic_mixed_locslope').quantile(0.9).values
        max_list.append(max_quad_semi_loc)
        ax[i].text(xmin,ymax,'Melt 90pctl = '+str(np.round(max_quad_semi_loc,1)))  
    elif i==3:
        melt_m_ice_per_y_PIGL_pattern.sel(param='lazero19_2').plot.contourf(ax=ax[i],cmap=cmaph,vmin=0,vmax=min_maxval,levels=levs,add_colorbar=False)#,vmin=minval, vmax=maxval,zorder=0)  
        max_plume_laz19 = melt_m_ice_per_y_PIGL_pattern.sel(param='lazero19_2').quantile(0.9).values
        max_list.append(max_plume_laz19)
        ax[i].text(xmin,ymax,'Melt 90pctl = '+str(np.round(max_plume_laz19,1)))         
    elif i==4:
        melt_m_ice_per_y_PIGL_pattern.sel(param='lazero19_modif2').plot.contourf(ax=ax[i],cmap=cmaph,vmin=0,vmax=min_maxval,levels=levs,add_colorbar=False)#,vmin=minval, vmax=maxval,zorder=0)  
        max_plume_modif = melt_m_ice_per_y_PIGL_pattern.sel(param='lazero19_modif2').quantile(0.9).values
        max_list.append(max_plume_modif)
        ax[i].text(xmin,ymax,'Melt 90pctl = '+str(np.round(max_plume_modif,1)))          
    elif i==5:
        melt_m_ice_per_y_PIGL_pattern.sel(param='boxes_3_pismyes_picopno').plot.contourf(ax=ax[i],cmap=cmaph,vmin=0,vmax=min_maxval,levels=levs,add_colorbar=False)#,vmin=minval, vmax=maxval,zorder=0)  
        max_boxes_3 = melt_m_ice_per_y_PIGL_pattern.sel(param='boxes_3_pismyes_picopno').quantile(0.9).values
        max_list.append(max_boxes_3)
        ax[i].text(xmin,ymax,'Melt 90pctl = '+str(np.round(max_boxes_3,1)))  
    elif i==6:
        #melt_m_ice_per_y_PIGL_pattern.sel(param='boxes_4_pismno_picopyes').plot.contourf(ax=ax[i],cmap=cmaph,vmin=0,vmax=min_maxval,levels=levs,extend='both')#,vmin=minval, vmax=maxval,zorder=0)  
        melt_m_ice_per_y_PIGL_pattern.sel(param='boxes_4_pismno_picopyes').plot.contourf(ax=ax[i],cmap=cmaph,vmin=0,vmax=min_maxval,levels=levs,add_colorbar=False)#,vmin=minval, vmax=maxval,zorder=0)  
        max_picop_4 = melt_m_ice_per_y_PIGL_pattern.sel(param='boxes_4_pismno_picopyes').quantile(0.9).values
        max_list.append(max_picop_4)
        ax[i].text(xmin,ymax,'Melt 90pctl = '+str(np.round(max_picop_4,1))) 
        
    ax[i].contour(ref_melt_isf.x,ref_melt_isf.y,grounded_msk.where(ref_melt_isf),levels=[0,1],linewidths=0.5,colors='black',zorder=10)
    #ax[i].contour(ref_melt_isf.x,ref_melt_isf.y,box1_msk.where(ref_melt_isf),levels=[0,1],linewidths=0.5,colors='blue',zorder=10)
    ax[i].set_xticks([])
    ax[i].set_xlabel('')
    ax[i].set_yticks([])
    ax[i].set_ylabel('')
    ax[i].set_title('')
    
      
    #ax[i].contour(ref_melt_isf.x,ref_melt_isf.y,icesheet_msk.where(ref_melt_isf),levels=[0,1],linewidths=0.5,colors='black',zorder=15)
#if np.max(max_list) > ref_melt_isf.max().values:
#    cbar = fig.colorbar(cax, extend='max')
#else:
#cbar = fig.colorbar(cax)
plt.tight_layout()
fig.savefig(plot_path+'comparison_spatial_patterns_PIGL_dutrieux'+BedMachine_folder+'_without_colorbar.png', dpi=300)

PLOT FIGURE 8C

In [ ]:
#try to normalise

kisf = 66


ref_melt_isf = melt_m_ice_per_y_PIGL_pattern.isel(param=0)
#cmaph = cmocean.cm.thermal
cmaph = mpl.cm.Reds

levs = np.arange(0,1.1,0.1)#[245, 250, 255, 260, 265, 270, 275, 280, 285, 290, 310, 340]

if kisf == 11:
    ymax = ref_melt_isf.y.min() - 80000
    xmin = ref_melt_isf.x.min() + 10000
elif kisf == 10:
    ymax = ref_melt_isf.y.min() - 80000
    xmin = ref_melt_isf.x.min() + 5000
elif kisf == 31:
    ymax = ref_melt_isf.y.min() - 20000
    xmin = ref_melt_isf.x.min() + 3000 
elif kisf == 44:
    ymax = ref_melt_isf.y.min() - 20000
    xmin = ref_melt_isf.x.min() + 3000 
elif kisf == 48:
    ymax = ref_melt_isf.y.min() - 15000
    xmin = ref_melt_isf.x.min() + 3000   
elif kisf == 66:
    ymax = ref_melt_isf.y.min() - 15000
    xmin = ref_melt_isf.x.min() + 1000   
    
#min_maxval = (abs(ref_melt_isf)).max().values
min_maxval=50

max_list = []

fig, ax = plt.subplots(1,7,sharex=True,sharey=True)
fig.set_size_inches(8.25*1.7, 8.25/4.5)
for i in range(len(ax)):
    if i==0:
        max_lin_Ant = melt_m_ice_per_y_PIGL_pattern.sel(param='linear_local').quantile(0.9).values
        norm_pattern = melt_m_ice_per_y_PIGL_pattern.sel(param='linear_local')/max_lin_Ant
    elif i==1:
        max_quad_Ant = melt_m_ice_per_y_PIGL_pattern.sel(param='quadratic_local').quantile(0.9).values
        norm_pattern = melt_m_ice_per_y_PIGL_pattern.sel(param='quadratic_local')/max_quad_Ant
    elif i==2:
        max_quad_semi_loc = melt_m_ice_per_y_PIGL_pattern.sel(param='quadratic_mixed_locslope').quantile(0.9).values
        norm_pattern = melt_m_ice_per_y_PIGL_pattern.sel(param='quadratic_mixed_locslope')/max_quad_semi_loc
    elif i==3:
        max_plume_laz19 = melt_m_ice_per_y_PIGL_pattern.sel(param='lazero19_2').quantile(0.9).values
        norm_pattern = melt_m_ice_per_y_PIGL_pattern.sel(param='lazero19_2')/max_plume_laz19
    elif i==4:
        max_plume_modif = melt_m_ice_per_y_PIGL_pattern.sel(param='lazero19_modif2').quantile(0.9).values
        norm_pattern = melt_m_ice_per_y_PIGL_pattern.sel(param='lazero19_modif2')/max_plume_modif        
    elif i==5:
        max_boxes_3 = melt_m_ice_per_y_PIGL_pattern.sel(param='boxes_3_pismyes_picopno').quantile(0.9).values
        norm_pattern = melt_m_ice_per_y_PIGL_pattern.sel(param='boxes_3_pismyes_picopno')/max_boxes_3  
    elif i==6:
        max_picop_4 = melt_m_ice_per_y_PIGL_pattern.sel(param='boxes_4_pismno_picopyes').quantile(0.9).values
        norm_pattern = melt_m_ice_per_y_PIGL_pattern.sel(param='boxes_4_pismno_picopyes')/max_picop_4  
        #melt_m_ice_per_y_PIGL_pattern.sel(param='boxes_4_pismno_picopyes').plot.contourf(ax=ax[i],cmap=cmaph,vmin=0,vmax=min_maxval,levels=levs,extend='both')#,vmin=minval, vmax=maxval,zorder=0)  
#    if i < 6:
#        norm_pattern.plot.contourf(ax=ax[i],cmap=cmaph,vmin=0,vmax=1,levels=levs,add_colorbar=False)#,vmin=minval, vmax=maxval,zorder=0)  
#    else:
#        norm_pattern.plot.contourf(ax=ax[i],cmap=cmaph,vmin=0,vmax=1,levels=levs,extend='both')#,vmin=minval, vmax=maxval,zorder=0)  
    norm_pattern.plot.contourf(ax=ax[i],cmap=cmaph,vmin=0,vmax=1,levels=levs,add_colorbar=False)#,vmin=minval, vmax=maxval,zorder=0)  

        
    ax[i].contour(ref_melt_isf.x,ref_melt_isf.y,grounded_msk.where(ref_melt_isf),levels=[0,1],linewidths=0.5,colors='black',zorder=10)
    #ax[i].contour(ref_melt_isf.x,ref_melt_isf.y,box1_msk.where(ref_melt_isf),levels=[0,1],linewidths=0.5,colors='blue',zorder=10)
    ax[i].set_xticks([])
    ax[i].set_xlabel('')
    ax[i].set_yticks([])
    ax[i].set_ylabel('')
    ax[i].set_title('')
    
      
    #ax[i].contour(ref_melt_isf.x,ref_melt_isf.y,icesheet_msk.where(ref_melt_isf),levels=[0,1],linewidths=0.5,colors='black',zorder=15)
#if np.max(max_list) > ref_melt_isf.max().values:
#    cbar = fig.colorbar(cax, extend='max')
#else:
#cbar = fig.colorbar(cax)
plt.tight_layout()
fig.savefig(plot_path+'comparison_spatial_patterns_PIGL_dutrieux'+BedMachine_folder+'norm_without_colorbar.png', dpi=300)